# MuJoCo in Multiverse Tutorial

## <span style="color:red">**Important!!! Navigate to the 'Multiverse' directory first**</span>

In [1]:
if [ "$(basename "$PWD")" == "Multiverse" ]; then
    echo "You are in the 'Multiverse' directory."
else
    echo "You are not in the 'Multiverse' directory, navigate to the 'Multiverse' directory."
    cd ..
    pwd
    if [ "$(basename "$PWD")" == "Multiverse" ]; then
        echo "You are now in the 'Multiverse' directory."
    else
        echo "You are still not in the 'Multiverse' directory..."
    fi
fi
MULTIVERSE_DIR=$PWD

You are not in the 'Multiverse' directory, navigate to the 'Multiverse' directory.
/media/giang/Storage/Multiverse
You are now in the 'Multiverse' directory.


## 0. Clone some repositories and build them <span style="color:red">(only for local installation)</span>

These repositories provide examples of robot, object, and environment models. We will be importing a few models from them.

### 1) Open a new terminal

In [2]:
# gnome-terminal --working-directory=$MULTIVERSE_DIR

### 2) Install these ROS packages (for visualization and control)

```bash
sudo apt-get update && sudo apt-get install -y \
  ros-${ROS_DISTRO}-xacro \
  ros-${ROS_DISTRO}-rviz \
  ros-${ROS_DISTRO}-joint-trajectory-controller \
  ros-${ROS_DISTRO}-rqt-robot-steering \
  ros-${ROS_DISTRO}-rqt-joint-trajectory-controller \
  ros-${ROS_DISTRO}-joint-state-controller \
  ros-${ROS_DISTRO}-joint-state-publisher-gui \
  ros-${ROS_DISTRO}-effort-controllers \
  ros-${ROS_DISTRO}-gripper-action-controller \
  ros-${ROS_DISTRO}-dwa-local-planner
```

### 3) Clone some ROS packages and build them

```bash
source multiverse_ws/devel/setup.bash
git clone https://github.com/Multiverse-Framework/Multiverse-Robots.git multiverse_ws/src/multiverse_robots
git clone https://github.com/Multiverse-Framework/Multiverse-World.git multiverse_ws/src/multiverse_world
git clone https://github.com/Multiverse-Framework/Multiverse-Objects.git multiverse_ws/src/multiverse_objects
git clone https://github.com/Multiverse-Framework/Multiverse-Demos.git multiverse_ws/src/multiverse_demos
cd multiverse_ws
rosdep install --from-paths src --ignore-src -r -y
catkin build
```

## A. Import a robot from URDF into MuJoCo

### 1. Create a tutorial workspace

In [3]:
rm -rf $MULTIVERSE_DIR/tutorial_ws
mkdir -p $MULTIVERSE_DIR/tutorial_ws/src

### 2. Create `my_robot_description` package that stores all descriptions of the robot

In [4]:
source $MULTIVERSE_DIR/multiverse_ws/devel/setup.bash

cd $MULTIVERSE_DIR/tutorial_ws/src
catkin_create_pkg my_robot_description
cd ..
catkin build
cd ..

Created file my_robot_description/package.xml
Created file my_robot_description/CMakeLists.txt
Successfully created files in /media/giang/Storage/Multiverse/tutorial_ws/src/my_robot_description. Please adjust the values in package.xml.
--------------------------------------------------------------------------------
Profile:                     default
Extending:             [env] /media/giang/Storage/Multiverse/multiverse_ws/devel:/opt/ros/noetic
Workspace:                   /media/giang/Storage/Multiverse/tutorial_ws
--------------------------------------------------------------------------------
Build Space:        [exists] /media/giang/Storage/Multiverse/tutorial_ws/build
Devel Space:        [exists] /media/giang/Storage/Multiverse/tutorial_ws/devel
Install Space:      [unused] /media/giang/Storage/Multiverse/tutorial_ws/install
Log Space:         [missing] /media/giang/Storage/Multiverse/tutorial_ws/logs
Source Space:       [exists] /media/giang/Storage/Multiverse/tutorial_ws/src
D

### 3. Use `multiverse_parser` to clean up and convert the robot from URDF to MJCF

In [5]:
multiverse_parser --help

usage: multiverse_parser [-h] --input INPUT --output OUTPUT
                         [--physics | --no-physics] [--visual | --no-visual]
                         [--collision | --no-collision]
                         [--keepusd | --no-keepusd]
                         [--collisionrgba COLLISIONRGBA [COLLISIONRGBA ...]]

Multiverse parser

options:
  -h, --help            show this help message and exit
  --input INPUT         Import scene description as (URDF, MJCF, WORLD or USD)
  --output OUTPUT       Export scene description as (URDF, MJCF, WORLD or USD)
  --physics, --no-physics
                        Whether to include physics properties or not
  --visual, --no-visual
                        Whether to include visual meshes or not
  --collision, --no-collision
                        Whether to include collision meshes or not
  --keepusd, --no-keepusd
                        Whether to keep the USD file or not
  --collisionrgba COLLISIONRGBA [COLLISIONRGBA ...]
                 

#### a) Clean up the URDF

##### Convert all *.dae to *.obj and repath the meshes
##### Use `--no-collision` to drop all collision meshes from URDF

In [6]:
source $MULTIVERSE_DIR/tutorial_ws/devel/setup.bash

TIAGO_DUAL_URDF=$(rospack find tiago_dual_description)/urdf/tiago_dual.urdf
MY_ROBOT_URDF=$(rospack find my_robot_description)/urdf/tiago_dual.urdf

multiverse_parser --input=$TIAGO_DUAL_URDF --output=$MY_ROBOT_URDF --no-collision

Create /media/giang/Storage/Multiverse/multiverse/build/blender/bin/3.6/python/lib/python3.10/site-packages/multiverse_parser/.cache/yOiE0rvwfb/tmp.usda
(  0.0000 sec |   0.0000 sec) Importing OBJ '/media/giang/Storage/Multiverse/multiverse_ws/src/multiverse_robots/pal_robotics/tiago_dual_description/urdf/tiago_dual/obj/base.obj'...
  (  0.0001 sec |   0.0000 sec) Parsing OBJ file...
    (  0.1586 sec |   0.1585 sec) Done, loading materials and images...
    (  0.1597 sec |   0.1596 sec) Done, building geometries (verts:10748 faces:21429 materials: 1 smoothgroups:0) ...
    (  0.2707 sec |   0.2706 sec) Done.
  (  0.2707 sec |   0.2707 sec) Finished importing: '/media/giang/Storage/Multiverse/multiverse_ws/src/multiverse_robots/pal_robotics/tiago_dual_description/urdf/tiago_dual/obj/base.obj'
Progress: 100.00%

USD export of '/media/giang/Storage/Multiverse/multiverse/build/blender/bin/3.6/python/lib/python3.10/site-packages/multiverse_parser/.cache/yOiE0rvwfb/tmp/usd/visual/base.usda'

  (  0.0001 sec |   0.0000 sec) Parsing OBJ file...
    (  0.0075 sec |   0.0074 sec) Done, loading materials and images...
    (  0.0079 sec |   0.0078 sec) Done, building geometries (verts:808 faces:1616 materials: 1 smoothgroups:0) ...
    (  0.0178 sec |   0.0177 sec) Done.
  (  0.0178 sec |   0.0178 sec) Finished importing: '/media/giang/Storage/Multiverse/multiverse_ws/src/multiverse_robots/pal_robotics/tiago_dual_description/urdf/tiago_dual/obj/head_1.obj'
Progress: 100.00%

USD export of '/media/giang/Storage/Multiverse/multiverse/build/blender/bin/3.6/python/lib/python3.10/site-packages/multiverse_parser/.cache/yOiE0rvwfb/tmp/usd/visual/head_1.usda' took 6.0 ms
(  0.0000 sec |   0.0000 sec) Importing OBJ '/media/giang/Storage/Multiverse/multiverse_ws/src/multiverse_robots/pal_robotics/tiago_dual_description/urdf/tiago_dual/obj/head_2.obj'...
  (  0.0001 sec |   0.0000 sec) Parsing OBJ file...
    (  0.0131 sec |   0.0130 sec) Done, loading materials and images...
    (  0.0135

Progress: 100.00%

USD export of '/media/giang/Storage/Multiverse/multiverse/build/blender/bin/3.6/python/lib/python3.10/site-packages/multiverse_parser/.cache/yOiE0rvwfb/tmp/usd/visual/gripper_finger_link.usda' took 5.1 ms
(  0.0000 sec |   0.0000 sec) Importing OBJ '/media/giang/Storage/Multiverse/multiverse_ws/src/multiverse_robots/pal_robotics/tiago_dual_description/urdf/tiago_dual/obj/base_ring.obj'...
  (  0.0001 sec |   0.0000 sec) Parsing OBJ file...
    (  0.0078 sec |   0.0077 sec) Done, loading materials and images...
    (  0.0082 sec |   0.0081 sec) Done, building geometries (verts:871 faces:1728 materials: 1 smoothgroups:0) ...
    (  0.0171 sec |   0.0170 sec) Done.
  (  0.0172 sec |   0.0171 sec) Finished importing: '/media/giang/Storage/Multiverse/multiverse_ws/src/multiverse_robots/pal_robotics/tiago_dual_description/urdf/tiago_dual/obj/base_ring.obj'
Progress: 100.00%

USD export of '/media/giang/Storage/Multiverse/multiverse/build/blender/bin/3.6/python/lib/python3.

    (  0.0001 sec |   0.0000 sec) OBJ Export path: '/media/giang/Storage/Multiverse/tutorial_ws/src/my_robot_description/urdf/tiago_dual/obj/caster_2.obj'
          (  0.0078 sec |   0.0076 sec) Finished writing geometry of 'caster_2'.
      (  0.0079 sec |   0.0078 sec) Finished exporting geometry, now exporting materials
      (  0.0081 sec |   0.0080 sec) OBJ Export Finished
Progress: 100.00%

USD import of '/media/giang/Storage/Multiverse/multiverse/build/blender/bin/3.6/python/lib/python3.10/site-packages/multiverse_parser/.cache/yOiE0rvwfb/tmp/usd/visual/caster_1.usda' took 1.5 ms
    (  0.0001 sec |   0.0000 sec) OBJ Export path: '/media/giang/Storage/Multiverse/tutorial_ws/src/my_robot_description/urdf/tiago_dual/obj/caster_1.obj'
          (  0.0097 sec |   0.0096 sec) Finished writing geometry of 'caster_1'.
      (  0.0098 sec |   0.0098 sec) Finished exporting geometry, now exporting materials
      (  0.0101 sec |   0.0100 sec) OBJ Export Finished
Progress: 100.00%

USD im

    (  0.0001 sec |   0.0000 sec) OBJ Export path: '/media/giang/Storage/Multiverse/tutorial_ws/src/my_robot_description/urdf/tiago_dual/obj/arm_6_wrist_2017.obj'
          (  0.0355 sec |   0.0354 sec) Finished writing geometry of 'arm_6_wrist_2017'.
      (  0.0356 sec |   0.0355 sec) Finished exporting geometry, now exporting materials
      (  0.0358 sec |   0.0357 sec) OBJ Export Finished
Progress: 100.00%

USD import of '/media/giang/Storage/Multiverse/multiverse/build/blender/bin/3.6/python/lib/python3.10/site-packages/multiverse_parser/.cache/yOiE0rvwfb/tmp/usd/visual/gripper_link.usda' took 5.1 ms
    (  0.0001 sec |   0.0000 sec) OBJ Export path: '/media/giang/Storage/Multiverse/tutorial_ws/src/my_robot_description/urdf/tiago_dual/obj/gripper_link.obj'
          (  0.0847 sec |   0.0846 sec) Finished writing geometry of 'gripper_link'.
      (  0.0848 sec |   0.0847 sec) Finished exporting geometry, now exporting materials
      (  0.0850 sec |   0.0849 sec) OBJ Export Finish

      (  0.0219 sec |   0.0218 sec) OBJ Export Finished
Progress: 100.00%

Remove /media/giang/Storage/Multiverse/multiverse/build/blender/bin/3.6/python/lib/python3.10/site-packages/multiverse_parser/.cache/yOiE0rvwfb


##### Test the URDF

In [7]:
source $MULTIVERSE_DIR/tutorial_ws/devel/setup.bash

MY_ROBOT_URDF=$(rospack find my_robot_description)/urdf/tiago_dual.urdf

tree $(dirname $MY_ROBOT_URDF)
cat $MY_ROBOT_URDF

/media/giang/Storage/Multiverse/tutorial_ws/src/my_robot_description/urdf
├── tiago_dual
│   └── obj
│       ├── antenna.mtl
│       ├── antenna.obj
│       ├── arm_1.mtl
│       ├── arm_1.obj
│       ├── arm_2.mtl
│       ├── arm_2.obj
│       ├── arm_3.mtl
│       ├── arm_3.obj
│       ├── arm_4.mtl
│       ├── arm_4.obj
│       ├── arm_5_wrist_2017.mtl
│       ├── arm_5_wrist_2017.obj
│       ├── arm_6_wrist_2017.mtl
│       ├── arm_6_wrist_2017.obj
│       ├── base.mtl
│       ├── base.obj
│       ├── base_ring.mtl
│       ├── base_ring.obj
│       ├── caster_1.mtl
│       ├── caster_1.obj
│       ├── caster_2.mtl
│       ├── caster_2.obj
│       ├── gripper_finger_link.mtl
│       ├── gripper_finger_link.obj
│       ├── gripper_link.mtl
│       ├── gripper_link.obj
│       ├── head_1.mtl
│       ├── head_1.obj
│       ├── head_2.mtl
│       ├── head_2.obj
│       ├── sick_tim551.mtl
│       ├── sick_tim551.obj
│       ├── srf05.mtl
│       ├── srf05.obj
│       ├── textures
│     

    <child link="caster_front_right_1_link"/>
  </joint>
  <link name="caster_front_right_1_link">
    <visual name="caster_1">
      <origin xyz="0.0 0.0 0.0" rpy="0.0 -0.0 0.0"/>
      <geometry>
        <mesh filename="package://my_robot_description/urdf/tiago_dual/obj/caster_1.obj" scale="1.0 1.0 1.0"/>
      </geometry>
    </visual>
  </link>
  <joint name="caster_front_right_2_joint" type="continuous">
    <origin xyz="-0.01600000075995922 0.0 -0.03999999910593033" rpy="-1.5707963267948966 0.0 0.0"/>
    <axis xyz="0 0 1"/>
    <parent link="caster_front_right_1_link"/>
    <child link="caster_front_right_2_link"/>
  </joint>
  <link name="caster_front_right_2_link">
    <visual name="caster_2">
      <origin xyz="0.0 0.0 0.0" rpy="0.0 -0.0 0.0"/>
      <geometry>
        <mesh filename="package://my_robot_description/urdf/tiago_dual/obj/caster_2.obj" scale="1.0 1.0 1.0"/>
      </geometry>
    </visual>
  </link>
  <joint name="caster_front_left_1_joint" type="continuous">
    

    <origin xyz="0.0 0.0 0.0" rpy="-1.5707963267948966 -0.0 -1.5707963267948966"/>
    <parent link="xtion_rgb_frame"/>
    <child link="xtion_rgb_optical_frame"/>
  </joint>
  <link name="xtion_rgb_optical_frame"/>
  <joint name="arm_left_1_joint" type="revolute">
    <origin xyz="0.02556000091135502 0.1899999976158142 -0.17100000381469727" rpy="3.141592653589793 -0.0 1.5707963267948966"/>
    <axis xyz="0 0 1"/>
    <parent link="torso_lift_link"/>
    <child link="arm_left_1_link"/>
    <limit effort="1000" lower="-1.1780972450961724" upper="1.5707963267948966" velocity="1000"/>
  </joint>
  <link name="arm_left_1_link">
    <visual name="arm_1">
      <origin xyz="0.0 0.0 0.0" rpy="3.141592653589793 -0.0 0.0"/>
      <geometry>
        <mesh filename="package://my_robot_description/urdf/tiago_dual/obj/arm_1.obj" scale="1.0 1.0000000000000004 1.0000000000000004"/>
      </geometry>
    </visual>
  </link>
  <joint name="arm_left_2_joint" type="revolute">
    <origin xyz="0.125 -0.01

    <visual name="gripper_finger_link">
      <origin xyz="0.0 0.0 0.0" rpy="0.0 -0.0 3.1415"/>
      <geometry>
        <mesh filename="package://my_robot_description/urdf/tiago_dual/obj/gripper_finger_link.obj" scale="1.000092649297317 0.9999073421179959 1.0"/>
      </geometry>
    </visual>
  </link>
  <joint name="gripper_left_grasping_frame_joint" type="fixed">
    <origin xyz="0.0 0.0 -0.12" rpy="1.5707926536217427 1.5707926535897934 3.141592653589793"/>
    <parent link="gripper_left_link"/>
    <child link="gripper_left_grasping_frame"/>
  </joint>
  <link name="gripper_left_grasping_frame"/>
  <joint name="arm_right_1_joint" type="revolute">
    <origin xyz="0.02556000091135502 -0.1899999976158142 -0.17100000381469727" rpy="0.0 0.0 -1.5707963267948966"/>
    <axis xyz="0 0 1"/>
    <parent link="torso_lift_link"/>
    <child link="arm_right_1_link"/>
    <limit effort="1000" lower="-1.1780972450961724" upper="1.5707963267948966" velocity="1000"/>
  </joint>
  <link name="arm_

    <axis xyz="-1 0 0"/>
    <parent link="gripper_right_link"/>
    <child link="gripper_right_left_finger_link"/>
    <limit effort="1000" lower="0.0" upper="0.04500000178813934" velocity="1000"/>
  </joint>
  <link name="gripper_right_left_finger_link">
    <visual name="gripper_finger_link">
      <origin xyz="0.0 0.0 0.0" rpy="0.0 -0.0 3.1415"/>
      <geometry>
        <mesh filename="package://my_robot_description/urdf/tiago_dual/obj/gripper_finger_link.obj" scale="1.000092649297317 0.9999073421179959 1.0"/>
      </geometry>
    </visual>
  </link>
  <joint name="gripper_right_grasping_frame_joint" type="fixed">
    <origin xyz="0.0 0.0 -0.12" rpy="1.5707926536217427 1.5707926535897934 3.141592653589793"/>
    <parent link="gripper_right_link"/>
    <child link="gripper_right_grasping_frame"/>
  </joint>
  <link name="gripper_right_grasping_frame"/>
  <joint name="torso_fixed_column_link_joint" type="fixed">
    <origin xyz="-0.062 0.0 0.195" rpy="0.0 -0.0 0.0"/>
    <parent li

In [8]:
source $MULTIVERSE_DIR/tutorial_ws/devel/setup.bash

MY_ROBOT_URDF=$(rospack find my_robot_description)/urdf/tiago_dual.urdf

pkill rosmaster # Kill rosmaster if exists
roslaunch multiverse_parser view_urdf.launch urdf:=$MY_ROBOT_URDF headless:=true & sleep 5 # Wait 5s before executing the next cell

[1] 95912
... logging to /home/giang/.ros/log/4f4f2e52-5ae6-11ee-b596-d183e98fb698/roslaunch-giangnguyen-95912.log
Checking log directory for disk usage. This may take a while.
Press Ctrl-C to interrupt
Done checking log file disk usage. Usage is <1GB.
]2;/media/giang/Storage/Multiverse/multiverse_ws/src/multiverse_core/multiverse_parser/launch/view_urdf.launch
started roslaunch server http://giangnguyen:38645/

SUMMARY

PARAMETERS
 * /robot_description: <?xml version="1....
 * /rosdistro: noetic
 * /rosversion: 1.16.0

NODES
  /
    joint_state_publisher (joint_state_publisher/joint_state_publisher)
    robot_state_publisher (robot_state_publisher/robot_state_publisher)

auto-starting new master
process[master]: started with pid [95938]
ROS_MASTER_URI=http://localhost:11311
]2;/media/giang/Storage/Multiverse/multiverse_ws/src/multiverse_core/multiverse_parser/launch/view_urdf.launch http://localhost:11311
setting /run_id to 4f4f2e52-5ae6-11ee-b596-d183e98fb698
process[rosout-1]: s

#### b) Convert the robot from URDF into MJCF

##### Use `--no-visual` to drop all visual meshes from URDF

In [9]:
source $MULTIVERSE_DIR/tutorial_ws/devel/setup.bash

TIAGO_DUAL_URDF=$(rospack find tiago_dual_description)/urdf/tiago_dual.urdf
MY_ROBOT_MJCF=$(rospack find my_robot_description)/mjcf/tiago_dual.xml

multiverse_parser --input=$TIAGO_DUAL_URDF --output=$MY_ROBOT_MJCF --no-visual

Create /media/giang/Storage/Multiverse/multiverse/build/blender/bin/3.6/python/lib/python3.10/site-packages/multiverse_parser/.cache/JwWKkfNGec/tmp.usda
Import finished in 0.0015 sec.
USD export of '/media/giang/Storage/Multiverse/multiverse/build/blender/bin/3.6/python/lib/python3.10/site-packages/multiverse_parser/.cache/JwWKkfNGec/tmp/usd/collision/base_collision.usda' took 3.2 ms
Import finished in 0.0002 sec.
USD export of '/media/giang/Storage/Multiverse/multiverse/build/blender/bin/3.6/python/lib/python3.10/site-packages/multiverse_parser/.cache/JwWKkfNGec/tmp/usd/collision/torso_lift_collision_core.usda' took 1.2 ms
Import finished in 0.0004 sec.
USD export of '/media/giang/Storage/Multiverse/multiverse/build/blender/bin/3.6/python/lib/python3.10/site-packages/multiverse_parser/.cache/JwWKkfNGec/tmp/usd/collision/torso_lift_collision_shoulder.usda' took 1.4 ms
Import finished in 0.0003 sec.
USD export of '/media/giang/Storage/Multiverse/multiverse/build/blender/bin/3.6/python/l

##### Test the MJCF

In [10]:
source $MULTIVERSE_DIR/tutorial_ws/devel/setup.bash

MY_ROBOT_MJCF=$(rospack find my_robot_description)/mjcf/tiago_dual.xml

cat $MY_ROBOT_MJCF

<?xml version="1.0" ?>
<mujoco model="tiago_dual">
	<compiler meshdir="tiago_dual/" texturedir="tiago_dual/obj/textures" angle="radian" autolimits="true" balanceinertia="true" boundmass="0.000001" boundinertia="0.000001"/>
	<default>
		<default class="visual">
			<geom contype="0" conaffinity="0"/>
		</default>
		<default class="collision">
			<geom rgba="1.0 0.0 0.0 0.5"/>
		</default>
	</default>
	<asset>
		<mesh name="base_collision_collision1d01d01d0" file="stl/base_collision1d01d01d0.stl"/>
		<mesh name="torso_lift_collision_core_collision1d01d01d0" file="stl/torso_lift_collision_core1d01d01d0.stl"/>
		<mesh name="torso_lift_collision_neck_collision1d01d01d0" file="stl/torso_lift_collision_neck1d01d01d0.stl"/>
		<mesh name="torso_lift_collision_shoulder_collision1d01d01d0" file="stl/torso_lift_collision_shoulder1d01d01d0.stl"/>
		<mesh name="head_1_collision_collision1d01d01d0" file="stl/head_1_collision1d01d01d0.stl"/>
		<mesh name="head_2_collision_collision1d01d01d0" file="stl/

											<geom name="arm_left_4_link_collision_0" pos="0.0 0.0 0.0" quat="1.0 0.0 0.0 0.0" class="collision" type="mesh" mesh="arm_4_collision_collision_1d0_1d0_1d0"/>
											<joint name="arm_left_4_joint" pos="2.775557561562892e-17 -1.7347234759768056e-17 -3.469446951953608e-18" type="hinge" range="-0.39269908169872414 2.356194490192345" axis="0.0 0.0 1.0"/>
											<body name="arm_left_5_link" pos="0.16200000000000014 -0.020000000000000018 -0.027000000000000007" quat="0.7071067811865476 5.551115123125785e-17 -0.7071067811865475 -5.551115123125789e-17">
												<geom name="arm_left_5_link_collision_0" pos="0.0 0.0 0.0" quat="1.0 0.0 0.0 0.0" class="collision" type="mesh" mesh="arm_5_collision_wrist_2017_collision_1d0_1d0_1d0"/>
												<joint name="arm_left_5_joint" pos="1.7347234759768022e-17 6.938893903907336e-18 -1.3877787807814454e-16" type="hinge" range="-2.0943951023931953 2.0943951023931953" axis="0.0 0.0 1.0"/>
												<body name="arm_left_6_link" pos="0.0

														<joint name="arm_right_7_joint" pos="0.0 -0.0 0.0" type="hinge" range="-2.0943951023931953 2.0943951023931953" axis="0.0 0.0 1.0"/>
														<body name="arm_right_tool_link" pos="0.0 0.0 0.04600000000000004" quat="-0.5 0.5 0.4999999999999999 0.5000000000000001">
															<geom name="arm_right_tool_link_collision_0" pos="0.001 0.0 0.0" quat="0.7071067811865476 0.0 0.7071067811865475 0.0" class="collision" type="cylinder" size="0.005 0.0025"/>
															<body name="wrist_right_ft_link" pos="0.007849999999999913 0.0 0.0" quat="0.5000000000000001 0.5 0.4999999999999999 0.5">
																<geom name="wrist_right_ft_link_collision_0" pos="0.0 0.0 0.0" quat="1.0 0.0 0.0 0.0" class="collision" type="cylinder" size="0.0225 0.00785"/>
																<body name="wrist_right_ft_tool_link" pos="0.0 0.0 0.012724999999999875" quat="0.5000000000000001 -0.5 -0.5 -0.4999999999999999">
																	<geom name="wrist_right_ft_tool_link_collision_0" pos="0.0 0.0 0.0" q

## B. Establish a connection between the robot in MuJoCo and ROS

### 1. Create `my_robot_mujoco` package to launch the robot in MuJoCo from ROS

#### a) Create the package and build it

In [11]:
source $MULTIVERSE_DIR/tutorial_ws/devel/setup.bash

cd $MULTIVERSE_DIR/tutorial_ws/src
catkin_create_pkg my_robot_mujoco
mkdir my_robot_mujoco/launch
mkdir my_robot_mujoco/config
touch my_robot_mujoco/config/mujoco_params.yaml
touch my_robot_mujoco/config/control_params.yaml
touch my_robot_mujoco/config/multiverse_params.yaml
touch my_robot_mujoco/launch/my_robot_mujoco.launch
cd ..
catkin build
cd ..

Created file my_robot_mujoco/package.xml
Created file my_robot_mujoco/CMakeLists.txt
Successfully created files in /media/giang/Storage/Multiverse/tutorial_ws/src/my_robot_mujoco. Please adjust the values in package.xml.
--------------------------------------------------------------------------------
Profile:                     default
Extending:          [cached] /media/giang/Storage/Multiverse/multiverse_ws/devel:/opt/ros/noetic
Workspace:                   /media/giang/Storage/Multiverse/tutorial_ws
--------------------------------------------------------------------------------
Build Space:        [exists] /media/giang/Storage/Multiverse/tutorial_ws/build
Devel Space:        [exists] /media/giang/Storage/Multiverse/tutorial_ws/devel
Install Space:      [unused] /media/giang/Storage/Multiverse/tutorial_ws/install
Log Space:          [exists] /media/giang/Storage/Multiverse/tutorial_ws/logs
Source Space:       [exists] /media/giang/Storage/Multiverse/tutorial_ws/src
DESTDIR:        

#### b) Copy the following configuration into `mujoco_params.yaml`

```yaml
mujoco:
  robots: [tiago_dual]
  pose_init: [1.5, 2.5, 0.0, 0.0, 0.0, 0.0]
  add_odom_joints: true
  root_frame_id: map
  
multiverse:  
  length_unit: m
  angle_unit: rad
  force_unit: N
  time_unit: s
  handedness: rhs
  
  send:
    body: [position, quaternion]
    joint: [joint_rvalue, joint_tvalue]
    
  receive:
    tiago_dual: [relative_velocity]
```

In [12]:
MUJOCO_PARAMS_YAML=$MULTIVERSE_DIR/tutorial_ws/src/my_robot_mujoco/config/mujoco_params.yaml

echo "mujoco:
  robots: [tiago_dual]
  pose_init: [1.5, 2.5, 0.0, 0.0, 0.0, 0.0]
  add_odom_joints: true
  root_frame_id: map
  
multiverse:  
  length_unit: m
  angle_unit: rad
  force_unit: N
  time_unit: s
  handedness: rhs

  send:
    body: [position, quaternion]
    joint: [joint_rvalue, joint_tvalue]

  receive:
    tiago_dual: [relative_velocity]" > $MUJOCO_PARAMS_YAML
  
cat $MUJOCO_PARAMS_YAML

mujoco:
  robots: [tiago_dual]
  pose_init: [1.5, 2.5, 0.0, 0.0, 0.0, 0.0]
  add_odom_joints: true
  root_frame_id: map
  
multiverse:  
  length_unit: m
  angle_unit: rad
  force_unit: N
  time_unit: s
  handedness: rhs

  send:
    body: [position, quaternion]
    joint: [joint_rvalue, joint_tvalue]

  receive:
    tiago_dual: [relative_velocity]


#### c) Copy the following configuration into `multiverse_params.yaml`

```yaml
multiverse:  
  length_unit: m
  angle_unit: rad
  force_unit: N
  time_unit: s
  handedness: rhs
  
  services:
    query_data:
      port: 7400

  publishers:
    tf:
      port: 7300
      rate: 60

  subscribers:
    cmd_vel:
      port: 7200
      body: tiago_dual
```

In [13]:
MULTIVERSE_PARAMS_YAML=$MULTIVERSE_DIR/tutorial_ws/src/my_robot_mujoco/config/multiverse_params.yaml

echo "multiverse:  
  length_unit: m
  angle_unit: rad
  force_unit: N
  time_unit: s
  handedness: rhs
  
  services:
    query_data:
      port: 7400

  publishers:
    tf:
      port: 7300
      rate: 60

  subscribers:
    cmd_vel:
      port: 7200
      body: tiago_dual" > $MULTIVERSE_PARAMS_YAML
  
cat $MULTIVERSE_PARAMS_YAML

multiverse:  
  length_unit: m
  angle_unit: rad
  force_unit: N
  time_unit: s
  handedness: rhs
  
  services:
    query_data:
      port: 7400

  publishers:
    tf:
      port: 7300
      rate: 60

  subscribers:
    cmd_vel:
      port: 7200
      body: tiago_dual


#### a) Copy the following configuration into `control_params.yaml`

```yaml
joint_state_controller:
  type: joint_state_controller/JointStateController
  publish_rate: 60

torso_controller:
  type: "effort_controllers/JointTrajectoryController"
  joints:
    - torso_lift_joint
  gains:
    torso_lift_joint: &pid_gains {p: 2000, i: 100, d: 50}

head_controller:
  type: "effort_controllers/JointTrajectoryController"
  joints:
    - head_1_joint
    - head_2_joint
  gains:
    head_1_joint: *pid_gains
    head_2_joint: *pid_gains

arm_left_controller:
  type: "effort_controllers/JointTrajectoryController"
  joints:
    - arm_left_1_joint
    - arm_left_2_joint
    - arm_left_3_joint
    - arm_left_4_joint
    - arm_left_5_joint
    - arm_left_6_joint
    - arm_left_7_joint
  gains:
    arm_left_1_joint: *pid_gains
    arm_left_2_joint: *pid_gains
    arm_left_3_joint: *pid_gains
    arm_left_4_joint: *pid_gains
    arm_left_5_joint: *pid_gains
    arm_left_6_joint: *pid_gains
    arm_left_7_joint: *pid_gains

arm_right_controller:
  type: "effort_controllers/JointTrajectoryController"
  joints:
    - arm_right_1_joint
    - arm_right_2_joint
    - arm_right_3_joint
    - arm_right_4_joint
    - arm_right_5_joint
    - arm_right_6_joint
    - arm_right_7_joint
  gains:
    arm_right_1_joint: *pid_gains
    arm_right_2_joint: *pid_gains
    arm_right_3_joint: *pid_gains
    arm_right_4_joint: *pid_gains
    arm_right_5_joint: *pid_gains
    arm_right_6_joint: *pid_gains
    arm_right_7_joint: *pid_gains

gripper_left_left_finger_controller:
  type: effort_controllers/JointEffortController
  joint: gripper_left_left_finger_joint
gripper_left_right_finger_controller:
  type: effort_controllers/JointEffortController
  joint: gripper_left_right_finger_joint
gripper_right_left_finger_controller:
  type: effort_controllers/JointEffortController
  joint: gripper_right_left_finger_joint
gripper_right_right_finger_controller:
  type: effort_controllers/JointEffortController
  joint: gripper_right_right_finger_joint
```

In [14]:
CONTROL_PARAMS_YAML=$MULTIVERSE_DIR/tutorial_ws/src/my_robot_mujoco/config/control_params.yaml

echo "joint_state_controller:
  type: joint_state_controller/JointStateController
  publish_rate: 60

torso_controller:
  type: \"effort_controllers/JointTrajectoryController\"
  joints:
    - torso_lift_joint
  gains:
    torso_lift_joint: &pid_gains {p: 2000, i: 100, d: 50}

head_controller:
  type: \"effort_controllers/JointTrajectoryController\"
  joints:
    - head_1_joint
    - head_2_joint
  gains:
    head_1_joint: *pid_gains
    head_2_joint: *pid_gains

arm_left_controller:
  type: \"effort_controllers/JointTrajectoryController\"
  joints:
    - arm_left_1_joint
    - arm_left_2_joint
    - arm_left_3_joint
    - arm_left_4_joint
    - arm_left_5_joint
    - arm_left_6_joint
    - arm_left_7_joint
  gains:
    arm_left_1_joint: *pid_gains
    arm_left_2_joint: *pid_gains
    arm_left_3_joint: *pid_gains
    arm_left_4_joint: *pid_gains
    arm_left_5_joint: *pid_gains
    arm_left_6_joint: *pid_gains
    arm_left_7_joint: *pid_gains

arm_right_controller:
  type: \"effort_controllers/JointTrajectoryController\"
  joints:
    - arm_right_1_joint
    - arm_right_2_joint
    - arm_right_3_joint
    - arm_right_4_joint
    - arm_right_5_joint
    - arm_right_6_joint
    - arm_right_7_joint
  gains:
    arm_right_1_joint: *pid_gains
    arm_right_2_joint: *pid_gains
    arm_right_3_joint: *pid_gains
    arm_right_4_joint: *pid_gains
    arm_right_5_joint: *pid_gains
    arm_right_6_joint: *pid_gains
    arm_right_7_joint: *pid_gains

gripper_left_left_finger_controller:
  type: effort_controllers/JointEffortController
  joint: gripper_left_left_finger_joint
gripper_left_right_finger_controller:
  type: effort_controllers/JointEffortController
  joint: gripper_left_right_finger_joint
gripper_right_left_finger_controller:
  type: effort_controllers/JointEffortController
  joint: gripper_right_left_finger_joint
gripper_right_right_finger_controller:
  type: effort_controllers/JointEffortController
  joint: gripper_right_right_finger_joint" > $CONTROL_PARAMS_YAML
  
cat $CONTROL_PARAMS_YAML

joint_state_controller:
  type: joint_state_controller/JointStateController
  publish_rate: 60

torso_controller:
  type: "effort_controllers/JointTrajectoryController"
  joints:
    - torso_lift_joint
  gains:
    torso_lift_joint: &pid_gains {p: 2000, i: 100, d: 50}

head_controller:
  type: "effort_controllers/JointTrajectoryController"
  joints:
    - head_1_joint
    - head_2_joint
  gains:
    head_1_joint: *pid_gains
    head_2_joint: *pid_gains

arm_left_controller:
  type: "effort_controllers/JointTrajectoryController"
  joints:
    - arm_left_1_joint
    - arm_left_2_joint
    - arm_left_3_joint
    - arm_left_4_joint
    - arm_left_5_joint
    - arm_left_6_joint
    - arm_left_7_joint
  gains:
    arm_left_1_joint: *pid_gains
    arm_left_2_joint: *pid_gains
    arm_left_3_joint: *pid_gains
    arm_left_4_joint: *pid_gains
    arm_left_5_joint: *pid_gains
    arm_left_6_joint: *pid_gains
    arm_left_7_joint: *pid_gains

arm_right_controller:
  type: "effort_controllers/Join

#### d) Copy this launch file into `my_robot_mujoco.launch`

```xml
<launch>

    <include file="$(find mujoco_sim)/launch/mujoco_sim_headless.launch"> <!-- Change to mujoco_sim.launch to see the MuJoCo world -->
        <arg name="config" value="$(find my_robot_mujoco)/config/mujoco_params.yaml" />
        <arg name="robot" value="$(find my_robot_description)/mjcf/tiago_dual.xml" />
    </include>
    
    <param name="robot_description" command="$(find xacro)/xacro '$(find my_robot_description)/urdf/tiago_dual.urdf'" />
    <rosparam command="load" file="$(find my_robot_mujoco)/config/control_params.yaml" />
    <node name="controller_manager" pkg="controller_manager" type="controller_manager" respawn="false" output="screen" args="spawn 
        joint_state_controller
        torso_controller
        head_controller
        arm_left_controller
        arm_right_controller
        gripper_left_left_finger_controller
        gripper_left_right_finger_controller
        gripper_right_right_finger_controller
        gripper_right_left_finger_controller" />
    <rosparam command="load" file="$(find my_robot_mujoco)/config/multiverse_params.yaml" />
    <node pkg="multiverse_socket" type="multiverse_socket_node.py" name="multiverse_socket" output="screen" />
    <!--node name="rqt_joint_trajectory_controller" pkg="rqt_joint_trajectory_controller" type="rqt_joint_trajectory_controller" />
    <node name="rqt_robot_steering" pkg="rqt_robot_steering" type="rqt_robot_steering" /-->

</launch>
```

In [15]:
MY_ROBOT_LAUNCH=$MULTIVERSE_DIR/tutorial_ws/src/my_robot_mujoco/launch/my_robot_mujoco.launch

echo "<launch>
    <include file=\"\$(find mujoco_sim)/launch/mujoco_sim_headless.launch\">
        <arg name=\"config\" value=\"\$(find my_robot_mujoco)/config/mujoco_params.yaml\" />
        <arg name=\"robot\" value=\"\$(find my_robot_description)/mjcf/tiago_dual.xml\" />
    </include>
    
    <param name=\"robot_description\" command=\"\$(find xacro)/xacro '\$(find my_robot_description)/urdf/tiago_dual.urdf'\" />
    <rosparam command=\"load\" file=\"\$(find my_robot_mujoco)/config/control_params.yaml\" />
    <node name=\"controller_manager\" pkg=\"controller_manager\" type=\"controller_manager\" respawn=\"false\" output=\"screen\" args=\"spawn 
        joint_state_controller\" />
    
    <rosparam command=\"load\" file=\"\$(find my_robot_mujoco)/config/multiverse_params.yaml\" />
    <node pkg=\"multiverse_socket\" type=\"multiverse_socket_node.py\" name=\"multiverse_socket\" output=\"screen\" />
</launch>" > $MY_ROBOT_LAUNCH
    
cat $MY_ROBOT_LAUNCH

<launch>
    <include file="$(find mujoco_sim)/launch/mujoco_sim_headless.launch">
        <arg name="config" value="$(find my_robot_mujoco)/config/mujoco_params.yaml" />
        <arg name="robot" value="$(find my_robot_description)/mjcf/tiago_dual.xml" />
    </include>
    
    <param name="robot_description" command="$(find xacro)/xacro '$(find my_robot_description)/urdf/tiago_dual.urdf'" />
    <rosparam command="load" file="$(find my_robot_mujoco)/config/control_params.yaml" />
    <node name="controller_manager" pkg="controller_manager" type="controller_manager" respawn="false" output="screen" args="spawn 
        joint_state_controller" />
    
    <rosparam command="load" file="$(find my_robot_mujoco)/config/multiverse_params.yaml" />
    <node pkg="multiverse_socket" type="multiverse_socket_node.py" name="multiverse_socket" output="screen" />
</launch>


#### e) Launch the robot in MuJoCo from ROS

In [16]:
source $MULTIVERSE_DIR/tutorial_ws/devel/setup.bash

pkill rosmaster && pkill multiverse_serv # Kill rosmaster and multiverse_server if exist
multiverse_server & roslaunch my_robot_mujoco my_robot_mujoco.launch & sleep 5 # Wait 5s before executing the next cell

================================================================================REQUIRED process [master] has died!
process has died [pid 95938, exit code -15, cmd rosmaster --core -p 11311 -w 3 __log:=/home/giang/.ros/log/4f4f2e52-5ae6-11ee-b596-d183e98fb698/master.log].
log file: /home/giang/.ros/log/4f4f2e52-5ae6-11ee-b596-d183e98fb698/master*.log
Initiating shutdown!
[2] 96471
[3] 96472
Start Multiverse Server...
[Server] Create server socket tcp://*:7000, waiting for client...
[joint_state_publisher-3] killing on exit
[robot_state_publisher-2] killing on exit
... logging to /home/giang/.ros/log/5a098eaa-5ae6-11ee-b596-d183e98fb698/roslaunch-giangnguyen-96472.log
Checking log directory for disk usage. This may take a while.
Press Ctrl-C to interrupt
Done checking log file disk usage. Usage is <1GB.
]2;/media/giang/Storage/Multiverse/tutorial_ws/src/my_robot_mujoco/launch/my_robot_mujoco.launch
started roslaunch server http://giangnguyen:36623/

SUMMARY

PARAMETERS
 * /arm_left_co

done
[ INFO] [1695565590.261959844]: Initialized the MuJoCo simulator successfully.
[ INFO] [1695565590.262069112]: Initializing the ROS interface...
[ INFO] [1695565590.266925712]: Started [/mujoco/screenshot] service.
[ INFO] [1695565590.267783727]: Started [/mujoco/reset] service.
[ INFO] [1695565590.268189951]: Started [/mujoco/spawn_objects] service.
[ INFO] [1695565590.269071496]: Started [/mujoco/destroy_objects] service.
[ WARN] [1695565590.269355481]: joint_inits not found, will set to default value (0)
[ INFO] [1695565590.270155738]: Initialized the ROS interface successfully.
[ INFO] [1695565590.271343778]: Set receive_objects: tiago_dual 
[ INFO] [1695565590.272467340]: Set send_objects: 
[Client 7500] Opened the socket tcp://127.0.0.1:7500.
[Server] Bind to socket tcp://*:7500.
Loaded 'joint_state_controller'
Started ['joint_state_controller'] successfully
[INFO] [1695565590.572605]: Start publisher [tf] with {'port': 7300, 'rate': 60}
[Client 7200] Opened the socket tcp:/

#### d) Insert this tag into my_robot.xml, before </mujoco> to exclude the contact pairs

```xml
<contact>
    <exclude body1="torso_lift_link" body2="torso_fixed_column_link"/>
    <exclude body1="base_link" body2="wheel_right_link"/>
    <exclude body1="base_link" body2="wheel_left_link"/>
    <exclude body1="base_link" body2="caster_back_left_2_link"/>
    <exclude body1="base_link" body2="caster_back_right_2_link"/>
    <exclude body1="base_link" body2="caster_front_left_2_link"/>
    <exclude body1="base_link" body2="caster_front_right_2_link"/>
</contact>
```


In [17]:
source $MULTIVERSE_DIR/tutorial_ws/devel/setup.bash

MY_ROBOT_MJCF=$(rospack find my_robot_description)/mjcf/tiago_dual.xml

sed -i '/<\/mujoco>/i \
    <contact>\
        <exclude body1="torso_lift_link" body2="torso_fixed_column_link"/>\
        <exclude body1="base_link" body2="wheel_right_link"/>\
        <exclude body1="base_link" body2="wheel_left_link"/>\
        <exclude body1="base_link" body2="caster_back_left_2_link"/>\
        <exclude body1="base_link" body2="caster_back_right_2_link"/>\
        <exclude body1="base_link" body2="caster_front_left_2_link"/>\
        <exclude body1="base_link" body2="caster_front_right_2_link"/>\
    </contact>' $MY_ROBOT_MJCF

cat $MY_ROBOT_MJCF

<?xml version="1.0" ?>
<mujoco model="tiago_dual">
	<compiler meshdir="tiago_dual/" texturedir="tiago_dual/obj/textures" angle="radian" autolimits="true" balanceinertia="true" boundmass="0.000001" boundinertia="0.000001"/>
	<default>
		<default class="visual">
			<geom contype="0" conaffinity="0"/>
		</default>
		<default class="collision">
			<geom rgba="1.0 0.0 0.0 0.5"/>
		</default>
	</default>
	<asset>
		<mesh name="base_collision_collision1d01d01d0" file="stl/base_collision1d01d01d0.stl"/>
		<mesh name="torso_lift_collision_core_collision1d01d01d0" file="stl/torso_lift_collision_core1d01d01d0.stl"/>
		<mesh name="torso_lift_collision_neck_collision1d01d01d0" file="stl/torso_lift_collision_neck1d01d01d0.stl"/>
		<mesh name="torso_lift_collision_shoulder_collision1d01d01d0" file="stl/torso_lift_collision_shoulder1d01d01d0.stl"/>
		<mesh name="head_1_collision_collision1d01d01d0" file="stl/head_1_collision1d01d01d0.stl"/>
		<mesh name="head_2_collision_collision1d01d01d0" file="stl/

											<geom name="arm_left_4_link_collision_0" pos="0.0 0.0 0.0" quat="1.0 0.0 0.0 0.0" class="collision" type="mesh" mesh="arm_4_collision_collision_1d0_1d0_1d0"/>
											<joint name="arm_left_4_joint" pos="2.775557561562892e-17 -1.7347234759768056e-17 -3.469446951953608e-18" type="hinge" range="-0.39269908169872414 2.356194490192345" axis="0.0 0.0 1.0"/>
											<body name="arm_left_5_link" pos="0.16200000000000014 -0.020000000000000018 -0.027000000000000007" quat="0.7071067811865476 5.551115123125785e-17 -0.7071067811865475 -5.551115123125789e-17">
												<geom name="arm_left_5_link_collision_0" pos="0.0 0.0 0.0" quat="1.0 0.0 0.0 0.0" class="collision" type="mesh" mesh="arm_5_collision_wrist_2017_collision_1d0_1d0_1d0"/>
												<joint name="arm_left_5_joint" pos="1.7347234759768022e-17 6.938893903907336e-18 -1.3877787807814454e-16" type="hinge" range="-2.0943951023931953 2.0943951023931953" axis="0.0 0.0 1.0"/>
												<body name="arm_left_6_link" pos="0.0

														<joint name="arm_right_7_joint" pos="0.0 -0.0 0.0" type="hinge" range="-2.0943951023931953 2.0943951023931953" axis="0.0 0.0 1.0"/>
														<body name="arm_right_tool_link" pos="0.0 0.0 0.04600000000000004" quat="-0.5 0.5 0.4999999999999999 0.5000000000000001">
															<geom name="arm_right_tool_link_collision_0" pos="0.001 0.0 0.0" quat="0.7071067811865476 0.0 0.7071067811865475 0.0" class="collision" type="cylinder" size="0.005 0.0025"/>
															<body name="wrist_right_ft_link" pos="0.007849999999999913 0.0 0.0" quat="0.5000000000000001 0.5 0.4999999999999999 0.5">
																<geom name="wrist_right_ft_link_collision_0" pos="0.0 0.0 0.0" quat="1.0 0.0 0.0 0.0" class="collision" type="cylinder" size="0.0225 0.00785"/>
																<body name="wrist_right_ft_tool_link" pos="0.0 0.0 0.012724999999999875" quat="0.5000000000000001 -0.5 -0.5 -0.4999999999999999">
																	<geom name="wrist_right_ft_tool_link_collision_0" pos="0.0 0.0 0.0" q

        <exclude body1="base_link" body2="wheel_left_link"/>
        <exclude body1="base_link" body2="caster_back_left_2_link"/>
        <exclude body1="base_link" body2="caster_back_right_2_link"/>
        <exclude body1="base_link" body2="caster_front_left_2_link"/>
        <exclude body1="base_link" body2="caster_front_right_2_link"/>
    </contact>
</mujoco>


#### d) Launch the robot without self-collision

In [18]:
source $MULTIVERSE_DIR/tutorial_ws/devel/setup.bash

pkill rosmaster && pkill multiverse_serv # Kill rosmaster and multiverse_server if exist
multiverse_server & roslaunch my_robot_mujoco my_robot_mujoco.launch & sleep 5 # Wait 5s before executing the next cell

[4] 96697
[5] 96698
Start Multiverse Server...
[Server] Create server socket tcp://*:7000, waiting for client...
================================================================================REQUIRED process [master] has died!
process has died [pid 96513, exit code -15, cmd rosmaster --core -p 11311 -w 3 __log:=/home/giang/.ros/log/5a098eaa-5ae6-11ee-b596-d183e98fb698/master.log].
log file: /home/giang/.ros/log/5a098eaa-5ae6-11ee-b596-d183e98fb698/master*.log
Initiating shutdown!
[multiverse_socket-4] killing on exit
[mujoco_sim-2] killing on exit
[Client 7500] Closing the socket tcp://127.0.0.1:7500.
... logging to /home/giang/.ros/log/5ddb4870-5ae6-11ee-b596-d183e98fb698/roslaunch-giangnguyen-96698.log
Checking log directory for disk usage. This may take a while.
Press Ctrl-C to interrupt
Done checking log file disk usage. Usage is <1GB.
]2;/media/giang/Storage/Multiverse/tutorial_ws/src/my_robot_mujoco/launch/my_robot_mujoco.launch
started roslaunch server http://giangnguyen:457

[ INFO] [1695565596.706546447]: Initializing the ROS interface...
[ INFO] [1695565596.711824099]: Started [/mujoco/screenshot] service.
[ INFO] [1695565596.712248042]: Started [/mujoco/reset] service.
[ INFO] [1695565596.713345544]: Started [/mujoco/spawn_objects] service.
[ INFO] [1695565596.713778817]: Started [/mujoco/destroy_objects] service.
[ WARN] [1695565596.714548914]: joint_inits not found, will set to default value (0)
[ INFO] [1695565596.715189884]: Initialized the ROS interface successfully.
[ INFO] [1695565596.716377854]: Set receive_objects: tiago_dual 
[ INFO] [1695565596.717371338]: Set send_objects: 
[Client 7500] Opened the socket tcp://127.0.0.1:7500.
[Server] Bind to socket tcp://*:7500.
Loaded 'joint_state_controller'
[INFO] [1695565596.976481]: Start publisher [tf] with {'port': 7300, 'rate': 60}
[Client 7200] Opened the socket tcp://127.0.0.1:7200.
[Client 7200] Start.
[Server] Bind to socket tcp://*:7200.
[Client 7200] Starting the communication (send: 7, recei

## C. Control the robot in MuJoCo using ros_control

### 1. Add controllers to arguments in `controller_manager`

```xml
joint_state_controller
torso_controller
head_controller
arm_left_controller
arm_right_controller
gripper_left_left_finger_controller
gripper_left_right_finger_controller
gripper_right_right_finger_controller
gripper_right_left_finger_controller
```

In [19]:
MY_ROBOT_LAUNCH=$MULTIVERSE_DIR/tutorial_ws/src/my_robot_mujoco/launch/my_robot_mujoco.launch

echo "<launch>
    <include file=\"\$(find mujoco_sim)/launch/mujoco_sim_headless.launch\">
        <arg name=\"config\" value=\"\$(find my_robot_mujoco)/config/mujoco_params.yaml\" />
        <arg name=\"robot\" value=\"\$(find my_robot_description)/mjcf/tiago_dual.xml\" />
    </include>
    
    <param name=\"robot_description\" command=\"\$(find xacro)/xacro '\$(find my_robot_description)/urdf/tiago_dual.urdf'\" />
    <rosparam command=\"load\" file=\"\$(find my_robot_mujoco)/config/control_params.yaml\" />
    <node name=\"controller_manager\" pkg=\"controller_manager\" type=\"controller_manager\" respawn=\"false\" output=\"screen\" args=\"spawn 
        joint_state_controller
        torso_controller
        head_controller
        arm_left_controller
        arm_right_controller
        gripper_left_left_finger_controller
        gripper_left_right_finger_controller
        gripper_right_right_finger_controller
        gripper_right_left_finger_controller\" />
    
    <rosparam command=\"load\" file=\"\$(find my_robot_mujoco)/config/multiverse_params.yaml\" />
    <node pkg=\"multiverse_socket\" type=\"multiverse_socket_node.py\" name=\"multiverse_socket\" output=\"screen\" />
</launch>" > $MY_ROBOT_LAUNCH
    
cat $MY_ROBOT_LAUNCH

<launch>
    <include file="$(find mujoco_sim)/launch/mujoco_sim_headless.launch">
        <arg name="config" value="$(find my_robot_mujoco)/config/mujoco_params.yaml" />
        <arg name="robot" value="$(find my_robot_description)/mjcf/tiago_dual.xml" />
    </include>
    
    <param name="robot_description" command="$(find xacro)/xacro '$(find my_robot_description)/urdf/tiago_dual.urdf'" />
    <rosparam command="load" file="$(find my_robot_mujoco)/config/control_params.yaml" />
    <node name="controller_manager" pkg="controller_manager" type="controller_manager" respawn="false" output="screen" args="spawn 
        joint_state_controller
        torso_controller
        head_controller
        arm_left_controller
        arm_right_controller
        gripper_left_left_finger_controller
        gripper_left_right_finger_controller
        gripper_right_right_finger_controller
        gripper_right_left_finger_controller" />
    
    <rosparam command="load" file="$(find my_robot_muj

### 2. Launch the robot with controllers

In [20]:
source $MULTIVERSE_DIR/tutorial_ws/devel/setup.bash

pkill rosmaster && pkill multiverse_serv # Kill rosmaster and multiverse_server if exist
multiverse_server & roslaunch my_robot_mujoco my_robot_mujoco.launch & sleep 5 # Wait 5s before executing the next cell

================================================================================REQUIRED process [master] has died!
process has died [pid 96740, exit code -15, cmd rosmaster --core -p 11311 -w 3 __log:=/home/giang/.ros/log/5ddb4870-5ae6-11ee-b596-d183e98fb698/master.log].
log file: /home/giang/.ros/log/5ddb4870-5ae6-11ee-b596-d183e98fb698/master*.log
Initiating shutdown!
[6] 96921
[7] 96922
Start Multiverse Server...
[Server] Create server socket tcp://*:7000, waiting for client...
[multiverse_socket-4] killing on exit
[mujoco_sim-2] killing on exit
[Client 7500] Closing the socket tcp://127.0.0.1:7500.
... logging to /home/giang/.ros/log/61e66b84-5ae6-11ee-b596-d183e98fb698/roslaunch-giangnguyen-96922.log
Checking log directory for disk usage. This may take a while.
Press Ctrl-C to interrupt
Done checking log file disk usage. Usage is <1GB.
]2;/media/giang/Storage/Multiverse/tutorial_ws/src/my_robot_mujoco/launch/my_robot_mujoco.launch
started roslaunch server http://giangnguyen:430

[ INFO] [1695565603.453769078]: Initializing the ROS interface...
[ INFO] [1695565603.465462455]: Started [/mujoco/screenshot] service.
[ INFO] [1695565603.465861368]: Started [/mujoco/reset] service.
[ INFO] [1695565603.466311891]: Started [/mujoco/spawn_objects] service.
[ INFO] [1695565603.466750144]: Started [/mujoco/destroy_objects] service.
[ WARN] [1695565603.467011189]: joint_inits not found, will set to default value (0)
[ INFO] [1695565603.467222256]: Initialized the ROS interface successfully.
[ INFO] [1695565603.468429426]: Set receive_objects: tiago_dual 
[ INFO] [1695565603.469435959]: Set send_objects: 
[Client 7500] Opened the socket tcp://127.0.0.1:7500.
[Server] Bind to socket tcp://*:7500.
Loaded 'joint_state_controller'
[INFO] [1695565603.730073]: Start publisher [tf] with {'port': 7300, 'rate': 60}
[Server] Bind to socket tcp://*:7200.
[Client 7200] Opened the socket tcp://127.0.0.1:7200.
[Client 7200] Start.
[Client 7200] Starting the communication (send: 7, recei

## D. Import an environment from USD into MuJoCo

### 1. Create `my_env_description` package that stores all descriptions of the environment

In [21]:
source $MULTIVERSE_DIR/multiverse_ws/devel/setup.bash

cd $MULTIVERSE_DIR/tutorial_ws/src
catkin_create_pkg my_env_description
cd ..
catkin build
cd ..

Created file my_env_description/package.xml
Created file my_env_description/CMakeLists.txt
Successfully created files in /media/giang/Storage/Multiverse/tutorial_ws/src/my_env_description. Please adjust the values in package.xml.
--------------------------------------------------------------------------------
Profile:                     default
Extending:          [cached] /media/giang/Storage/Multiverse/multiverse_ws/devel:/opt/ros/noetic
Workspace:                   /media/giang/Storage/Multiverse/tutorial_ws
--------------------------------------------------------------------------------
Build Space:        [exists] /media/giang/Storage/Multiverse/tutorial_ws/build
Devel Space:        [exists] /media/giang/Storage/Multiverse/tutorial_ws/devel
Install Space:      [unused] /media/giang/Storage/Multiverse/tutorial_ws/install
Log Space:          [exists] /media/giang/Storage/Multiverse/tutorial_ws/logs
Source Space:       [exists] /media/giang/Storage/Multiverse/tutorial_ws/src
DESTDIR

### 2. Use `multiverse_parser` to convert the environment from USD into URDF and into MJCF

#### a) Check the environment in USD

In [22]:
source $MULTIVERSE_DIR/tutorial_ws/devel/setup.bash

MY_ENV_IN_USD=$(rospack find iai_apartment)/usd/apartment.usda
cat $MY_ENV_IN_USD

#usda 1.0
(
    defaultPrim = "apartment_root"
    metersPerUnit = 1
    upAxis = "Z"
)

def Xform "apartment_root"
{
    def Xform "envelope_root"
    {
        matrix4d xformOp:transform = ( (1, 0, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0), (8.85, 1.75, 0, 1) )
        uniform token[] xformOpOrder = ["xformOp:transform"]

        def Xform "wall1"
        {
            matrix4d xformOp:transform = ( (1, 0, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0), (0, 0, -0.2, 1) )
            uniform token[] xformOpOrder = ["xformOp:transform"]

            def Xform "wall1_visual_0" (
                prepend references = @./../meshes/usd/visual/SM_Wall_01.usda@</SM_Wall_01>
            )
            {
                matrix4d xformOp:transform = ( (1, 0, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0), (0, 0, 0, 1) )
                uniform token[] xformOpOrder = ["xformOp:transform"]

                over "SM_Wall_01" (
                    prepend apiSchemas = ["MaterialBindingAPI"]
                )
                {
       

                uniform token[] xformOpOrder = ["xformOp:transform"]

                def Cube "Cube" (
                    prepend apiSchemas = ["PhysicsCollisionAPI"]
                )
                {
                    float3[] extent = [(-1, -1, -1), (1, 1, 1)]
                    bool physics:collisionEnabled = 1
                    color3f[] primvars:displayColor = [(1, 0, 0)]
                    float[] primvars:displayOpacity = [0.5]
                }
            }
        }

        def Xform "wall4"
        {
            matrix4d xformOp:transform = ( (1, 0, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0), (0, 0, -0.2, 1) )
            uniform token[] xformOpOrder = ["xformOp:transform"]

            def Xform "wall4_visual_0" (
                prepend references = @./../meshes/usd/visual/SM_Wall_04.usda@</SM_Wall_04>
            )
            {
                matrix4d xformOp:transform = ( (1, 0, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0), (0, 0, 0, 1) )
                uniform token[] xformOp

                matrix4d xformOp:transform = ( (0.01, 0, 0, 0), (0, 1.0677, 0, 0), (0, 0, 0.026796, 0), (0, 0, -1.49481, 1) )
                uniform token[] xformOpOrder = ["xformOp:transform"]

                def Cube "Cube" (
                    prepend apiSchemas = ["PhysicsCollisionAPI"]
                )
                {
                    float3[] extent = [(-1, -1, -1), (1, 1, 1)]
                    bool physics:collisionEnabled = 1
                    color3f[] primvars:displayColor = [(1, 0, 0)]
                    float[] primvars:displayOpacity = [0.5]
                }
            }

            def Xform "window4_frame_collision_3" (
                prepend apiSchemas = ["PhysicsMassAPI"]
            )
            {
                point3f physics:centerOfMass = (0, 0, 0)
                float physics:density = 100
                float3 physics:diagonalInertia = (-339.10748, -158.49792, 820.45416)
                float physics:mass = 3.2243605
                quatf p

        {
            matrix4d xformOp:transform = ( (1, 0, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0), (0.067235, 0, -0.057755, 1) )
            uniform token[] xformOpOrder = ["xformOp:transform"]

            def Xform "window4_right_handle_visual_0" (
                prepend references = @./../meshes/usd/visual/SM_Window_Handle.usda@</SM_Window_Handle>
            )
            {
                matrix4d xformOp:transform = ( (1, 0, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0), (0, 0, 0, 1) )
                uniform token[] xformOpOrder = ["xformOp:transform"]

                over "SM_Window_Handle_001" (
                    prepend apiSchemas = ["MaterialBindingAPI"]
                )
                {
                    rel material:binding = </_materials/_Game_Materials_Laborraum_M_Metal_Chrome_M_Metal_Chrome_011>
                }
            }

            def Xform "window4_right_handle_collision_0" (
                prepend apiSchemas = ["PhysicsMassAPI"]
            )
            {
         

                )
                {
                    rel material:binding = </_materials/_Game_Materials_Laborraum_M_Metal_Chrome_M_Metal_Chrome_011>
                }
            }

            def Xform "window4_left_handle_collision_0" (
                prepend apiSchemas = ["PhysicsMassAPI"]
            )
            {
                point3f physics:centerOfMass = (0, 0, 0)
                float physics:density = 100
                float3 physics:diagonalInertia = (-51.959843, -25.92772, 96.734955)
                float physics:mass = 1.9810693
                quatf physics:principalAxes = (0.004201073, 0.99949497, -0.030256249, 0.008759642)
                matrix4d xformOp:transform = ( (2.220446049250313e-16, 0, -1, 0), (0, 1, 0, 0), (1, 0, 2.220446049250313e-16, 0), (-0.029618, 0, 0.057814, 1) )
                uniform token[] xformOpOrder = ["xformOp:transform"]

                def Cylinder "Cylinder" (
                    prepend apiSchemas = ["PhysicsCollisionAPI"]
    

...


            def Cube "Cube" (
                prepend apiSchemas = ["PhysicsCollisionAPI"]
            )
            {
                float3[] extent = [(-1, -1, -1), (1, 1, 1)]
                bool physics:collisionEnabled = 1
                color3f[] primvars:displayColor = [(1, 0, 0)]
                float[] primvars:displayOpacity = [0.5]
            }
        }

        def Xform "cabinet10_drawer3_collision_5" (
            prepend apiSchemas = ["PhysicsMassAPI"]
        )
        {
            point3f physics:centerOfMass = (0, 0, 0)
            float physics:density = 100
            float3 physics:diagonalInertia = (-2.645346, -2.4815633, 7.133539)
            float physics:mass = 0.750759
            quatf physics:principalAxes = (-0.5118642, -0.4610361, 0.72486675, 0.0029947215)
            matrix4d xformOp:transform = ( (0.008, 0, 0, 0), (0, 0.178822, 0, 0), (0, 0, 0.084105, 0), (0.260372, 0, -0.088821, 1) )
            uniform token[] xformOpOrder = ["xformOp:transform"

                    prepend apiSchemas = ["MaterialBindingAPI"]
                )
                {
                    rel material:binding = </_materials/_Game_Materials_Laborraum_M_Plastic_White_M_Plastic_White_017>
                }

                over "_Game_Materials_Laborraum_M_Drawer_Mat_M_Drawer_Mat_008" (
                    prepend apiSchemas = ["MaterialBindingAPI"]
                )
                {
                    rel material:binding = </_materials/_Game_Materials_Laborraum_M_Drawer_Mat_M_Drawer_Mat_008>
                }
            }
        }

        def Xform "cabinet11_drawer1_collision_0" (
            prepend apiSchemas = ["PhysicsMassAPI"]
        )
        {
            point3f physics:centerOfMass = (0, 0, 0)
            float physics:density = 100
            float3 physics:diagonalInertia = (-10.994816, -6.510247, 18.374786)
            float physics:mass = 0.9771583
            quatf physics:principalAxes = (-0.2888746, -0.25373927, 0.33756083, 0.859

                uniform token[] xformOpOrder = ["xformOp:transform"]

                def Cube "Cube" (
                    prepend apiSchemas = ["PhysicsCollisionAPI"]
                )
                {
                    float3[] extent = [(-1, -1, -1), (1, 1, 1)]
                    bool physics:collisionEnabled = 1
                    color3f[] primvars:displayColor = [(1, 0, 0)]
                    float[] primvars:displayOpacity = [0.5]
                }
            }

            def Xform "cabinet11_drawer1_handle_collision_2" (
                prepend apiSchemas = ["PhysicsMassAPI"]
            )
            {
                point3f physics:centerOfMass = (0, 0, 0)
                float physics:density = 100
                float3 physics:diagonalInertia = (-10.994816, -6.510247, 18.374786)
                float physics:mass = 0.9771583
                quatf physics:principalAxes = (-0.2888746, -0.25373927, 0.33756083, 0.8591976)
                matrix4d xformOp:transform =

            matrix4d xformOp:transform = ( (0.0095, 0, 0, 0), (0, 0.448341, 0, 0), (0, 0, 0.131375, 0), (-0.233977, 0, 0, 1) )
            uniform token[] xformOpOrder = ["xformOp:transform"]

            def Cube "Cube" (
                prepend apiSchemas = ["PhysicsCollisionAPI"]
            )
            {
                float3[] extent = [(-1, -1, -1), (1, 1, 1)]
                bool physics:collisionEnabled = 1
                color3f[] primvars:displayColor = [(1, 0, 0)]
                float[] primvars:displayOpacity = [0.5]
            }
        }

        def Xform "cabinet11_drawer2_collision_5" (
            prepend apiSchemas = ["PhysicsMassAPI"]
        )
        {
            point3f physics:centerOfMass = (0, 0, 0)
            float physics:density = 100
            float3 physics:diagonalInertia = (-13.585582, -8.863193, 24.854698)
            float physics:mass = 1.3389739
            quatf physics:principalAxes = (0.36004195, 0.8541277, 0.28411108, 0.24518679)
     

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



#### b) Convert the environment from USD into URDF and into MJCF

In [23]:
source $MULTIVERSE_DIR/tutorial_ws/devel/setup.bash

MY_ENV_IN_USD=$(rospack find iai_apartment)/usd/apartment.usda
MY_ENV_IN_URDF=$(rospack find my_env_description)/urdf/my_environment.urdf
MY_ENV_IN_MJCF=$(rospack find my_env_description)/mjcf/my_environment.xml

multiverse_parser --input=$MY_ENV_IN_USD --output=$MY_ENV_IN_URDF --no-collision
multiverse_parser --input=$MY_ENV_IN_USD --output=$MY_ENV_IN_MJCF --no-visual

[multiverse_socket-4] escalating to SIGKILL
[rosout-1] killing on exit
[master] killing on exit
Shutdown errors:
 * process[multiverse_socket-4, pid 96549]: required SIGKILL. May still be running.
shutting down processing monitor...
... shutting down processing monitor complete
done
Create /media/giang/Storage/Multiverse/multiverse/build/blender/bin/3.6/python/lib/python3.10/site-packages/multiverse_parser/.cache/MDpmXzW3Qp/tmp.usda
USD import of '/media/giang/Storage/Multiverse/multiverse_ws/src/multiverse_world/iai_apartment/meshes/usd/visual/SM_Wall_01.usda' took 1.5 ms
USD export of '/media/giang/Storage/Multiverse/multiverse/build/blender/bin/3.6/python/lib/python3.10/site-packages/multiverse_parser/.cache/MDpmXzW3Qp/tmp/usd/visual/SM_Wall_01.usda' took 4.8 ms
USD import of '/media/giang/Storage/Multiverse/multiverse_ws/src/multiverse_world/iai_apartment/meshes/usd/visual/SM_Wall_02.usda' took 0.7 ms
USD export of '/media/giang/Storage/Multiverse/multiverse/build/blender/bin/3.6/p

USD export of '/media/giang/Storage/Multiverse/multiverse/build/blender/bin/3.6/python/lib/python3.10/site-packages/multiverse_parser/.cache/MDpmXzW3Qp/tmp/usd/visual/SM_Nachttisch.usda' took 3.3 ms
USD import of '/media/giang/Storage/Multiverse/multiverse_ws/src/multiverse_world/iai_apartment/meshes/usd/visual/wardrobe_door_left.usda' took 11.3 ms
Copying texture from /media/giang/Storage/Multiverse/multiverse_ws/src/multiverse_world/iai_apartment/meshes/usd/visual/textures/M_Metal_Chrome_M_Metal_Chrome_D.jpg to /media/giang/Storage/Multiverse/multiverse/build/blender/bin/3.6/python/lib/python3.10/site-packages/multiverse_parser/.cache/MDpmXzW3Qp/tmp/usd/visual/textures/M_Metal_Chrome_M_Metal_Chrome_D.jpg
Copying texture from /media/giang/Storage/Multiverse/multiverse_ws/src/multiverse_world/iai_apartment/meshes/usd/visual/textures/M_Wood_Furniture_M_Wood_Furniture_D.jpg to /media/giang/Storage/Multiverse/multiverse/build/blender/bin/3.6/python/lib/python3.10/site-packages/multiverse_

USD import of '/media/giang/Storage/Multiverse/multiverse_ws/src/multiverse_world/iai_apartment/meshes/usd/visual/SM_Kitchen_02_Base.usda' took 16.6 ms
Copying texture from /media/giang/Storage/Multiverse/multiverse_ws/src/multiverse_world/iai_apartment/meshes/usd/visual/textures/M_Metal_Chrome_M_Metal_Chrome_D.jpg to /media/giang/Storage/Multiverse/multiverse/build/blender/bin/3.6/python/lib/python3.10/site-packages/multiverse_parser/.cache/MDpmXzW3Qp/tmp/usd/visual/textures/M_Metal_Chrome_M_Metal_Chrome_D.jpg
Copying texture from /media/giang/Storage/Multiverse/multiverse_ws/src/multiverse_world/iai_apartment/meshes/usd/visual/textures/M_Rubber_Plastic_M_Rubber_Plastic_D.jpg to /media/giang/Storage/Multiverse/multiverse/build/blender/bin/3.6/python/lib/python3.10/site-packages/multiverse_parser/.cache/MDpmXzW3Qp/tmp/usd/visual/textures/M_Rubber_Plastic_M_Rubber_Plastic_D.jpg
Copying texture from /media/giang/Storage/Multiverse/multiverse_ws/src/multiverse_world/iai_apartment/meshes/u

Copying texture from /media/giang/Storage/Multiverse/multiverse_ws/src/multiverse_world/iai_apartment/meshes/usd/visual/textures/M_Metal_Chrome_M_Metal_Chrome_D.jpg to /media/giang/Storage/Multiverse/multiverse/build/blender/bin/3.6/python/lib/python3.10/site-packages/multiverse_parser/.cache/MDpmXzW3Qp/tmp/usd/visual/textures/M_Metal_Chrome_M_Metal_Chrome_D.jpg
Copying texture from /media/giang/Storage/Multiverse/multiverse_ws/src/multiverse_world/iai_apartment/meshes/usd/visual/textures/M_Kitchen_Fronts_M_Kitchen_Fronts_D.jpg to /media/giang/Storage/Multiverse/multiverse/build/blender/bin/3.6/python/lib/python3.10/site-packages/multiverse_parser/.cache/MDpmXzW3Qp/tmp/usd/visual/textures/M_Kitchen_Fronts_M_Kitchen_Fronts_D.jpg
USD export of '/media/giang/Storage/Multiverse/multiverse/build/blender/bin/3.6/python/lib/python3.10/site-packages/multiverse_parser/.cache/MDpmXzW3Qp/tmp/usd/visual/SM_Kitchen_04_Door_02.usda' took 31.6 ms
USD import of '/media/giang/Storage/Multiverse/multive

USD export of '/media/giang/Storage/Multiverse/multiverse/build/blender/bin/3.6/python/lib/python3.10/site-packages/multiverse_parser/.cache/MDpmXzW3Qp/tmp/usd/visual/SM_Counter_Sink_001.usda' took 29.0 ms
USD import of '/media/giang/Storage/Multiverse/multiverse_ws/src/multiverse_world/iai_apartment/meshes/usd/visual/SM_Counter_Watertab.usda' took 3.9 ms
Copying texture from /media/giang/Storage/Multiverse/multiverse_ws/src/multiverse_world/iai_apartment/meshes/usd/visual/textures/M_Metal_Chrome_Matte_M_Metal_Chrome_Matte_D.jpg to /media/giang/Storage/Multiverse/multiverse/build/blender/bin/3.6/python/lib/python3.10/site-packages/multiverse_parser/.cache/MDpmXzW3Qp/tmp/usd/visual/textures/M_Metal_Chrome_Matte_M_Metal_Chrome_Matte_D.jpg
USD export of '/media/giang/Storage/Multiverse/multiverse/build/blender/bin/3.6/python/lib/python3.10/site-packages/multiverse_parser/.cache/MDpmXzW3Qp/tmp/usd/visual/SM_Counter_Watertab.usda' took 33.6 ms
USD import of '/media/giang/Storage/Multiverse/

...

          (  0.0620 sec |   0.0618 sec) Finished writing geometry of 'SM_Kitchen_Handle'.
      (  0.0622 sec |   0.0621 sec) Finished exporting geometry, now exporting materials
      (  0.0625 sec |   0.0624 sec) OBJ Export Finished
Progress: 100.00%

USD import of '/media/giang/Storage/Multiverse/multiverse/build/blender/bin/3.6/python/lib/python3.10/site-packages/multiverse_parser/.cache/MDpmXzW3Qp/tmp/usd/visual/SM_Kitchen_02_Drawer_01.usda' took 7.3 ms
    (  0.0001 sec |   0.0000 sec) OBJ Export path: '/media/giang/Storage/Multiverse/tutorial_ws/src/my_env_description/urdf/my_environment/obj/SM_Kitchen_02_Drawer_01.obj'
          (  0.2227 sec |   0.2225 sec) Finished writing geometry of 'SM_Kitchen_02_Drawer_01'.
      (  0.2228 sec |   0.2227 sec) Finished exporting geometry, now exporting materials
      (  0.2233 sec |   0.2232 sec) OBJ Export Finished
Progress: 100.00%

USD import of '/media/giang/Storage/Multiverse/multiverse/build/blender/bin/3.6/python/lib/python3.10/sit

          (  0.0867 sec |   0.0865 sec) Finished writing geometry of 'SM_Kitchen_Handle'.
      (  0.0868 sec |   0.0868 sec) Finished exporting geometry, now exporting materials
      (  0.0872 sec |   0.0871 sec) OBJ Export Finished
Progress: 100.00%

USD import of '/media/giang/Storage/Multiverse/multiverse/build/blender/bin/3.6/python/lib/python3.10/site-packages/multiverse_parser/.cache/MDpmXzW3Qp/tmp/usd/visual/SM_Fridge_Drawer_01_001.usda' took 8.6 ms
    (  0.0001 sec |   0.0000 sec) OBJ Export path: '/media/giang/Storage/Multiverse/tutorial_ws/src/my_env_description/urdf/my_environment/obj/SM_Fridge_Drawer_01_001.obj'
          (  0.2557 sec |   0.2555 sec) Finished writing geometry of 'SM_Fridge_Drawer_01_001'.
      (  0.2558 sec |   0.2557 sec) Finished exporting geometry, now exporting materials
      (  0.2562 sec |   0.2561 sec) OBJ Export Finished
Progress: 100.00%

USD import of '/media/giang/Storage/Multiverse/multiverse/build/blender/bin/3.6/python/lib/python3.10/sit

          (  0.0651 sec |   0.0648 sec) Finished writing geometry of 'SM_Kitchen_Handle'.
      (  0.0652 sec |   0.0651 sec) Finished exporting geometry, now exporting materials
      (  0.0658 sec |   0.0657 sec) OBJ Export Finished
Progress: 100.00%

USD import of '/media/giang/Storage/Multiverse/multiverse/build/blender/bin/3.6/python/lib/python3.10/site-packages/multiverse_parser/.cache/MDpmXzW3Qp/tmp/usd/visual/SM_Kitchen_06_Base_Drawer_02.usda' took 8.1 ms
    (  0.0001 sec |   0.0000 sec) OBJ Export path: '/media/giang/Storage/Multiverse/tutorial_ws/src/my_env_description/urdf/my_environment/obj/SM_Kitchen_06_Base_Drawer_02.obj'
          (  0.2803 sec |   0.2801 sec) Finished writing geometry of 'SM_Kitchen_06_Base_Drawer_02'.
      (  0.2803 sec |   0.2802 sec) Finished exporting geometry, now exporting materials
      (  0.2809 sec |   0.2808 sec) OBJ Export Finished
Progress: 100.00%

USD import of '/media/giang/Storage/Multiverse/multiverse/build/blender/bin/3.6/python/lib

          (  0.0633 sec |   0.0631 sec) Finished writing geometry of 'SM_Kitchen_Handle'.
      (  0.0635 sec |   0.0634 sec) Finished exporting geometry, now exporting materials
      (  0.0638 sec |   0.0637 sec) OBJ Export Finished
Progress: 100.00%

USD import of '/media/giang/Storage/Multiverse/multiverse/build/blender/bin/3.6/python/lib/python3.10/site-packages/multiverse_parser/.cache/MDpmXzW3Qp/tmp/usd/visual/SM_Kitchen_09_Base_Drawer_03.usda' took 8.5 ms
    (  0.0001 sec |   0.0000 sec) OBJ Export path: '/media/giang/Storage/Multiverse/tutorial_ws/src/my_env_description/urdf/my_environment/obj/SM_Kitchen_09_Base_Drawer_03.obj'
          (  0.2754 sec |   0.2752 sec) Finished writing geometry of 'SM_Kitchen_09_Base_Drawer_03'.
      (  0.2755 sec |   0.2754 sec) Finished exporting geometry, now exporting materials
      (  0.2761 sec |   0.2760 sec) OBJ Export Finished
Progress: 100.00%

USD import of '/media/giang/Storage/Multiverse/multiverse/build/blender/bin/3.6/python/lib

          (  0.2775 sec |   0.2773 sec) Finished writing geometry of 'SM_Kitchen_11_Drawer_03'.
      (  0.2775 sec |   0.2774 sec) Finished exporting geometry, now exporting materials
      (  0.2781 sec |   0.2780 sec) OBJ Export Finished
Progress: 100.00%

USD import of '/media/giang/Storage/Multiverse/multiverse/build/blender/bin/3.6/python/lib/python3.10/site-packages/multiverse_parser/.cache/MDpmXzW3Qp/tmp/usd/visual/SM_Kitchen_Handle.usda' took 3.4 ms
    (  0.0001 sec |   0.0000 sec) OBJ Export path: '/media/giang/Storage/Multiverse/tutorial_ws/src/my_env_description/urdf/my_environment/obj/SM_Kitchen_Handle.obj'
          (  0.0899 sec |   0.0896 sec) Finished writing geometry of 'SM_Kitchen_Handle'.
      (  0.0900 sec |   0.0899 sec) Finished exporting geometry, now exporting materials
      (  0.0904 sec |   0.0903 sec) OBJ Export Finished
Progress: 100.00%

Remove /media/giang/Storage/Multiverse/multiverse/build/blender/bin/3.6/python/lib/python3.10/site-packages/multivers

#### d) Check the environment in URDF and MJCF

In [24]:
source $MULTIVERSE_DIR/tutorial_ws/devel/setup.bash

MY_ENV_IN_URDF=$(rospack find my_env_description)/urdf/my_environment.urdf

pkill rosmaster # Kill rosmaster if exists
roslaunch multiverse_parser view_urdf.launch urdf:=$MY_ENV_IN_URDF headless:=true & sleep 5 # Wait 5s before executing the next cell

[8] 98561
================================================================================REQUIRED process [master] has died!
process has died [pid 96963, exit code -15, cmd rosmaster --core -p 11311 -w 3 __log:=/home/giang/.ros/log/61e66b84-5ae6-11ee-b596-d183e98fb698/master.log].
log file: /home/giang/.ros/log/61e66b84-5ae6-11ee-b596-d183e98fb698/master*.log
Initiating shutdown!
[multiverse_socket-4] killing on exit
[mujoco_sim-2] killing on exit
[Client 7300] Closing the socket tcp://127.0.0.1:7300.
[Server] Received close signal from socket tcp://*:7300.
[Client 7500] Closing the socket tcp://127.0.0.1:7500.
[Server] Received close signal from socket tcp://*:7500.
... logging to /home/giang/.ros/log/7fb9da60-5ae6-11ee-b596-d183e98fb698/roslaunch-giangnguyen-98561.log
Checking log directory for disk usage. This may take a while.
Press Ctrl-C to interrupt
Done checking log file disk usage. Usage is <1GB.
]2;/media/giang/Storage/Multiverse/multiverse_ws/src/multiverse_core/multiverse

### 3. Use the environment in MuJoCo

#### a) Replace the following code in `my_robot_mujoco.launch`

```xml
<param name="env_description" command="$(find xacro)/xacro '$(find my_env_description)/urdf/my_environment.urdf'" />
<include file="$(find mujoco_sim)/launch/mujoco_sim_headless.launch"> <!-- Change to mujoco_sim.launch to see the MuJoCo world -->
    <arg name="config" value="$(find my_robot_mujoco)/config/mujoco_params.yaml" />
    <arg name="robot" value="$(find my_robot_description)/mjcf/my_robot.xml" />
    <arg name="world" value="$(find my_env_description)/mjcf/my_environment.xml" /> <!-- Add environment -->
</include>
```

In [25]:
MY_ROBOT_LAUNCH=$MULTIVERSE_DIR/tutorial_ws/src/my_robot_mujoco/launch/my_robot_mujoco.launch

echo "<launch>
    <param name=\"env_description\" command=\"\$(find xacro)/xacro '\$(find my_env_description)/urdf/my_environment.urdf'\" />
    <include file=\"\$(find mujoco_sim)/launch/mujoco_sim_headless.launch\">
        <arg name=\"config\" value=\"\$(find my_robot_mujoco)/config/mujoco_params.yaml\" />
        <arg name=\"robot\" value=\"\$(find my_robot_description)/mjcf/tiago_dual.xml\" />
        <arg name=\"world\" value=\"\$(find my_env_description)/mjcf/my_environment.xml\" />
    </include>
    
    <param name=\"robot_description\" command=\"\$(find xacro)/xacro '\$(find my_robot_description)/urdf/tiago_dual.urdf'\" />
    <rosparam command=\"load\" file=\"\$(find my_robot_mujoco)/config/control_params.yaml\" />
    <node name=\"controller_manager\" pkg=\"controller_manager\" type=\"controller_manager\" respawn=\"false\" output=\"screen\" args=\"spawn 
        joint_state_controller
        torso_controller
        head_controller
        arm_left_controller
        arm_right_controller
        gripper_left_left_finger_controller
        gripper_left_right_finger_controller
        gripper_right_right_finger_controller
        gripper_right_left_finger_controller\" />
    
    <rosparam command=\"load\" file=\"\$(find my_robot_mujoco)/config/multiverse_params.yaml\" />
    <node pkg=\"multiverse_socket\" type=\"multiverse_socket_node.py\" name=\"multiverse_socket\" output=\"screen\" />
</launch>" > $MY_ROBOT_LAUNCH
    
cat $MY_ROBOT_LAUNCH

<launch>
    <param name="env_description" command="$(find xacro)/xacro '$(find my_env_description)/urdf/my_environment.urdf'" />
    <include file="$(find mujoco_sim)/launch/mujoco_sim_headless.launch">
        <arg name="config" value="$(find my_robot_mujoco)/config/mujoco_params.yaml" />
        <arg name="robot" value="$(find my_robot_description)/mjcf/tiago_dual.xml" />
        <arg name="world" value="$(find my_env_description)/mjcf/my_environment.xml" />
    </include>
    
    <param name="robot_description" command="$(find xacro)/xacro '$(find my_robot_description)/urdf/tiago_dual.urdf'" />
    <rosparam command="load" file="$(find my_robot_mujoco)/config/control_params.yaml" />
    <node name="controller_manager" pkg="controller_manager" type="controller_manager" respawn="false" output="screen" args="spawn 
        joint_state_controller
        torso_controller
        head_controller
        arm_left_controller
        arm_right_controller
        gripper_left_left_finger_c

#### c) Launch the robot with environment in Multiverse (keep `multiverse_server` running)

In [26]:
source $MULTIVERSE_DIR/tutorial_ws/devel/setup.bash

pkill rosmaster && pkill multiverse_serv # Kill rosmaster and multiverse_server if exist
multiverse_server & roslaunch my_robot_mujoco my_robot_mujoco.launch & sleep 5 # Wait 5s before executing the next cell

================================================================================REQUIRED process [master] has died!
process has died [pid 98598, exit code -15, cmd rosmaster --core -p 11311 -w 3 __log:=/home/giang/.ros/log/7fb9da60-5ae6-11ee-b596-d183e98fb698/master.log].
log file: /home/giang/.ros/log/7fb9da60-5ae6-11ee-b596-d183e98fb698/master*.log
Initiating shutdown!
[9] 98716
[10] 98717
Start Multiverse Server...
[Server] Create server socket tcp://*:7000, waiting for client...
[joint_state_publisher-3] killing on exit
[robot_state_publisher-2] killing on exit
... logging to /home/giang/.ros/log/83cfa1fc-5ae6-11ee-b596-d183e98fb698/roslaunch-giangnguyen-98717.log
Checking log directory for disk usage. This may take a while.
Press Ctrl-C to interrupt
[rosout-1] killing on exit
Done checking log file disk usage. Usage is <1GB.
]2;/media/giang/Storage/Multiverse/tutorial_ws/src/my_robot_mujoco/launch/my_robot_mujoco.launch
[master] killing on exit
shutting down processing monitor..

[ INFO] [1695565660.484623310]: Initialized the MuJoCo simulator successfully.
[ INFO] [1695565660.484742238]: Initializing the ROS interface...
[ INFO] [1695565660.490340326]: Started [/mujoco/screenshot] service.
[ INFO] [1695565660.490803379]: Started [/mujoco/reset] service.
[ INFO] [1695565660.491907141]: Started [/mujoco/spawn_objects] service.
[ INFO] [1695565660.492346194]: Started [/mujoco/destroy_objects] service.
[ WARN] [1695565660.493175430]: joint_inits not found, will set to default value (0)
[ INFO] [1695565660.496046513]: Initialized the ROS interface successfully.
[ INFO] [1695565660.497474220]: Set receive_objects: tiago_dual 
[ INFO] [1695565660.498998904]: Set send_objects: 
[Server] Bind to socket tcp://*:7500.
[Client 7500] Opened the socket tcp://127.0.0.1:7500.
Loaded 'joint_state_controller'
[INFO] [1695565660.743359]: Start publisher [tf] with {'port': 7300, 'rate': 60}
[Client 7200] Opened the socket tcp://127.0.0.1:7200.
[Server] Bind to socket tcp://*:7200

In [27]:
pkill rosmaster && pkill multiverse_serv # Kill rosmaster and multiverse_server

: 1